通常

In [ ]:
import os
import time
import google.generativeai as genai
from PIL import Image, ImageDraw, ImageFont
import json
import typing_extensions as typing
from google.api_core import exceptions

# ---------------------------------------------------------
# 設定
# ---------------------------------------------------------

# ★前回成功したモデルを最優先に設定
PRIMARY_MODEL = "models/gemini-flash-latest"
# バックアップ
FALLBACK_MODEL = "gemini-1.5-flash"

INPUT_DIR = "./test_images"
OUTPUT_DIR = "./results_improved" # フォルダ名を変更して区別

target_images_list = [
    "sample2.png",
    "sample.png",
    "sample1.png",
    
]

# ---------------------------------------------------------

def extract_marks_high_accuracy(image_path, output_dir, api_key):
    file_name = os.path.basename(image_path)
    base_name = os.path.splitext(file_name)[0]
    
    genai.configure(api_key=api_key)
    
    # 構造化出力の定義
    class Box2D(typing.TypedDict):
        ymin: int
        xmin: int
        ymax: int
        xmax: int

    class MarkItem(typing.TypedDict):
        mark_type: str
        description: str # ここに日本語の説明が入る
        confidence: str  # 確信度 (High/Medium/Low) を追加
        box_2d: Box2D

    # 画像読み込み & RGB変換
    if not os.path.exists(image_path):
        print(f"[{file_name}] スキップ: ファイルなし")
        return

    try:
        img = Image.open(image_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')
    except Exception as e:
        print(f"[{file_name}] 画像読み込みエラー: {e}")
        return

    # ★★★ 改良されたプロンプト ★★★
    prompt = """
    この画像内の手書きの採点マーク（回答の正誤を示すマーク）を全て検出してください。
    対象: 丸 (Circle), バツ (Cross), 三角 (Triangle), チェック (Checkmark)。

    【重要な指示】
    1. **日本語で出力**: description（説明）は必ず日本語で記述してください（例: "問1(1)の回答", "大問2の採点欄"）。
    2. **正確な枠**: box_2d は、マークのインク部分ギリギリを囲むように、できるだけ余白を小さく（Tightly）検出してください。
    3. **文脈理解**: そのマークが「どの問題番号」に対するものか、周囲の文字を読み取って特定してください。
    4. **確信度**: 判定の自信を confidence に "High", "Medium", "Low" で記入してください。
    """

    # --- 実行ロジック ---
    models_to_try = [PRIMARY_MODEL, FALLBACK_MODEL]
    marks_data = None
    success_model = None

    print(f"--- [{file_name}] 高精度解析開始 ---")

    for model_name in models_to_try:
        if success_model: break
        
        # print(f"   [試行] モデル: {model_name}")
        
        model = genai.GenerativeModel(
            model_name=model_name,
            generation_config={
                "response_mime_type": "application/json", 
                "response_schema": list[MarkItem],
                "temperature": 0.0
            }
        )

        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = model.generate_content([prompt, img])
                marks_data = json.loads(response.text)
                success_model = model_name
                print(f"   -> 成功！ (モデル: {model_name})")
                break 

            except exceptions.NotFound:
                # print(f"   -> モデル {model_name} が見つかりません。")
                break
            
            except Exception as e:
                error_msg = str(e)
                if "limit: 0" in error_msg:
                    break 
                if "429" in error_msg:
                    wait_time = 20
                    print(f"   [待機] 混雑中... {wait_time}秒待機 ({attempt + 1}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"   -> エラー: {e}")
                    break

    if marks_data is None:
        print(f"   [失敗] データを取得できませんでした。")
        return

    print(f"   -> 検出数: {len(marks_data)} 個")

    # 可視化処理 (日本語フォント対応)
    if marks_data:
        draw = ImageDraw.Draw(img)
        width, height = img.size
        
        # フォント設定 (日本語を表示するためにデフォルト以外のフォントを探す)
        font = None
        # 一般的な日本語フォントのパス候補 (OSによって異なります)
        font_candidates = [
            "C:\\Windows\\Fonts\\msgothic.ttc", # Windows
            "/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc", # Mac
            "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc", # Linux
            "arial.ttf" # Fallback (日本語は豆腐になります)
        ]
        
        for f_path in font_candidates:
            try:
                font = ImageFont.truetype(f_path, 20)
                break
            except:
                continue

        for item in marks_data:
            box = item['box_2d']
            label = f"{item['mark_type']} ({item['confidence']})"
            desc = item['description'] # 日本語の説明
            
            abs_ymin = (box['ymin'] / 1000) * height
            abs_xmin = (box['xmin'] / 1000) * width
            abs_ymax = (box['ymax'] / 1000) * height
            abs_xmax = (box['xmax'] / 1000) * width

            # 確信度が低い場合は枠の色を変える (黄色)
            outline_color = "red"
            if item.get('confidence') == "Low":
                outline_color = "orange"

            draw.rectangle([(abs_xmin, abs_ymin), (abs_xmax, abs_ymax)], outline=outline_color, width=3)
            
            if font:
                # マーク種類と説明を表示
                display_text = f"{label}: {desc}"
                text_pos = (abs_xmin, max(0, abs_ymin - 25))
                
                # 背景帯を描画して文字を見やすく
                try:
                    bbox = draw.textbbox(text_pos, display_text, font=font)
                    draw.rectangle(bbox, fill=outline_color)
                    draw.text(text_pos, display_text, fill="white", font=font)
                except:
                    pass # フォント周りでエラーが出ても枠線だけは描画する

    # 保存処理
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    out_img_path = os.path.join(output_dir, f"{base_name}_improved.jpg")
    img.save(out_img_path, "JPEG", quality=95)
    
    out_json_path = os.path.join(output_dir, f"{base_name}_improved.json")
    with open(out_json_path, "w", encoding="utf-8") as f:
        json.dump(marks_data, f, ensure_ascii=False, indent=2)

    print(f"   -> 保存完了: {out_img_path}")

if __name__ == '__main__':
    # --- 実行設定 ---
    API_KEY = "APIAキーをここに貼り付けてください" 

    print(f"対象枚数: {len(target_images_list)} 枚")
    print("------------------------------------------------")
        
    for filename in target_images_list:
        full_path = os.path.join(INPUT_DIR, filename)
        extract_marks_high_accuracy(full_path, OUTPUT_DIR, API_KEY)
        
        time.sleep(2) 
            
    print("\nすべての処理が完了しました。")